In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import seaborn as sns
pd.options.display.max_rows = 1000

note: this notebook heavily leverages [this tutorial](https://www.kaggle.com/nieyuqi/mta-turnstile-data-analysis) as a starting ground for analysis.

In [2]:
mta = pd.read_csv("../data/Turnstiles/Turnstile_Usage_Data__2020.csv")
mta.columns = ['C/A', 'unit', 'SCP', 'station', 'line_name', 'division', 'date',
       'time', 'description', 'entries','exits']

In [3]:
# create unique turnstile column
mta['datetime'] = pd.to_datetime(mta.date + ' ' + mta.time, format='%m/%d/%Y  %H:%M:%S')
mta['turnstile'] = mta['C/A'] + '-' + mta.unit + '-' + mta['SCP']
mta["station_code"] = mta ["C/A"] + mta.unit

In [4]:
mta.head()

,C/A,unit,SCP,station,line_name,division,date,time,description,entries,exits,datetime,turnstile,station_code
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/24/2020,00:00:00,REGULAR,7415358,2517956,2020-04-24 00:00:00,A002-R051-02-00-00,A002R051
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/24/2020,04:00:00,REGULAR,7415359,2517956,2020-04-24 04:00:00,A002-R051-02-00-00,A002R051
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/24/2020,08:00:00,REGULAR,7415366,2517968,2020-04-24 08:00:00,A002-R051-02-00-00,A002R051
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/24/2020,12:00:00,REGULAR,7415374,2517990,2020-04-24 12:00:00,A002-R051-02-00-00,A002R051
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/24/2020,16:00:00,REGULAR,7415404,2518006,2020-04-24 16:00:00,A002-R051-02-00-00,A002R051


## 1.1 Calculate Differenced Data
`entries` and `exits` are both cumulative values that count up from the previous reading. We need to normalize these to get the incremental difference for each given time period. 

In [5]:
# group data by turnstile, sort each turnstile by datetime
# Create new columns en_diff and ex_diff for each unique turnstile
# turn cumulative counts into counts per interval

mta_sorted = mta.sort_values(["turnstile", "datetime"])
mta_sorted = mta_sorted.reset_index(drop = True)

In [6]:
turnstile_grouped = mta_sorted.groupby(["turnstile"])

In [7]:
mta_sorted['entries_diff'] = turnstile_grouped['entries'].transform(pd.Series.diff)
mta_sorted['exits_diff'] = turnstile_grouped['exits'].transform(pd.Series.diff)

In [8]:
print(f'Number of unqiue turnstiles: {len(mta_sorted.turnstile.unique())}') 
print(f'Number of NaN rows: {len(mta_sorted[mta_sorted.entries_diff.isnull()])}') 

Number of unqiue turnstiles: 4983
Number of NaN rows: 4983


In [9]:
print('summary of entries_diff:')
print(mta_sorted.entries_diff.describe())
print('99th percentile of entries_diff:')
entries_99th = mta_sorted.entries_diff.quantile(.9975)
print(entries_99th)

summary of entries_diff:
count    4.116365e+06
mean     1.979902e+03
std      2.521701e+06
min     -1.562921e+09
25%      0.000000e+00
50%      1.400000e+01
75%      9.300000e+01
max      2.038596e+09
Name: entries_diff, dtype: float64
99th percentile of entries_diff:
1390.0


In [10]:
print('summary of exits_diff:')
print(mta_sorted.exits_diff.describe())
print('99th percentile of exits_diff:')
exits_99th = mta_sorted.exits_diff.quantile(.9975)
print(exits_99th)

summary of exits_diff:
count    4.116365e+06
mean     1.696104e+03
std      2.655511e+06
min     -2.048960e+09
25%      0.000000e+00
50%      1.300000e+01
75%      7.100000e+01
max      2.036331e+09
Name: exits_diff, dtype: float64
99th percentile of exits_diff:
1395.0


In [11]:
# we saw that there was 1 N/A for each turnstile (the first in the dataset) so we'll set those values to zero
mta_sorted.exits_diff = mta_sorted.exits_diff.fillna(0)
mta_sorted.entries_diff = mta_sorted.entries_diff.fillna(0)

In [12]:
# these values should never be negative (since they are counting the increased counts each time), therefore we are cleaning out the negative values
mta_sorted.entries_diff[mta_sorted.entries_diff < 0] = 0
mta_sorted.exits_diff[mta_sorted.exits_diff < 0] = 0

# note: this probably happens when the turnstile cumulative counter is reset

In [13]:
# remove extreme values from dataset (anything aboove 99.75% of the sample)
mta_sorted.entries_diff[mta_sorted.entries_diff> entries_99th] = 0
mta_sorted.exits_diff[mta_sorted.exits_diff> exits_99th] = 0


1. see num hours from previous reading
2. divide entries/exits by number of hours
3. upsample data to have at hourly level
4. pad with normalized values (back-filling values)

In [14]:
# get the time difference from previous timestamp
mta_sorted['time_diff'] = turnstile_grouped['datetime'].transform(pd.Series.diff)
# turn it into a float so we can use it to normalize our aggregated values 
mta_sorted.time_diff = np.floor(mta_sorted.time_diff/np.timedelta64(1, 'h'))

In [15]:
# normalize our values to get the average hourly rate of entries and exits 
mta_sorted['entries_diff_hourly'] = mta_sorted.entries_diff / mta_sorted.time_diff
mta_sorted['exits_diff_hourly'] = mta_sorted.exits_diff / mta_sorted.time_diff

In [16]:
mta_sorted.head()

,C/A,unit,SCP,station,line_name,division,date,time,description,entries,exits,datetime,turnstile,station_code,entries_diff,exits_diff,time_diff,entries_diff_hourly,exits_diff_hourly
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/28/2019,03:00:00,REGULAR,7324295,2482512,2019-12-28 03:00:00,A002-R051-02-00-00,A002R051,0.0,0.0,NaN,NaN,NaN
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/28/2019,07:00:00,REGULAR,7324305,2482523,2019-12-28 07:00:00,A002-R051-02-00-00,A002R051,10.0,11.0,4.0,2.5,2.75
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/28/2019,11:00:00,REGULAR,7324371,2482594,2019-12-28 11:00:00,A002-R051-02-00-00,A002R051,66.0,71.0,4.0,16.5,17.75
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/28/2019,15:00:00,REGULAR,7324587,2482647,2019-12-28 15:00:00,A002-R051-02-00-00,A002R051,216.0,53.0,4.0,54.0,13.25
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/28/2019,19:00:00,REGULAR,7324963,2482713,2019-12-28 19:00:00,A002-R051-02-00-00,A002R051,376.0,66.0,4.0,94.0,16.50


In [17]:
# upsample data to have at hourly level
# time_resampled = mta_sorted.set_index('datetime')
time_grouped = mta_sorted.groupby(["turnstile"]).apply(lambda x : x.drop_duplicates('datetime')
.set_index('datetime')
.resample('1H')
.bfill())
# ref: https://stackoverflow.com/a/39793110
# ref: https://towardsdatascience.com/how-to-interpolate-time-series-data-in-apache-spark-and-python-pandas-part-1-pandas-cff54d76a2ea

In [18]:
time_resampled = time_grouped.drop("turnstile", axis=1).reset_index()

In [20]:
time_resampled.entries_diff_hourly = time_resampled.entries_diff_hourly.fillna(0).replace([np.inf, -np.inf], 0)
time_resampled.exits_diff_hourly = time_resampled.exits_diff_hourly.fillna(0).replace([np.inf, -np.inf], 0)


In [24]:
time_resampled.entries_diff_hourly.describe()

count    1.404909e+07
mean     2.570474e+01
std      4.511989e+01
min      0.000000e+00
25%      7.500000e-01
50%      6.000000e+00
75%      2.975000e+01
max      1.060000e+03
Name: entries_diff_hourly, dtype: float64